In [1]:
%load_ext autoreload
%autoreload 2

# Specify HyperParameters

In [2]:
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.2"
device = "cuda:0"
dataset_name = "../workdir/data/triviaqa.csv"
batch_size = 2
seed = 42

# Initialize Model

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    load_in_8bit=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token

generation_config = GenerationConfig.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
messages = [
    [
        {
            "role": "user", 
            "content": "How many fingers on a coala's foot?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Who sang a song Yesterday?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Кто спел песню Кукла Колдуна?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Translate into French: 'I want a small cup of coffee'"
        }
    ]
]

chat_messages = [tokenizer.apply_chat_template(m, tokenize=False) for m in messages]

# Infer LLM and get uncertainty scores

In [5]:
from lm_polygraph.stat_calculators.basic_probs import BasicGreedyProbsCalculatorCausalLM
from lm_polygraph.stat_calculators.greedy_alternatives_nli import GreedyTokensAlternativesNLICalculator
from lm_polygraph.estimators.claim_conditioned_probability import ClaimConditionedProbability
from lm_polygraph.estimators.common import CommonDeberta

from torch.utils.data import DataLoader


model_wrapper = BasicGreedyProbsCalculatorCausalLM()

nli_model = CommonDeberta(device=device)
nli_model.setup()
calc_nli = GreedyTokensAlternativesNLICalculator(nli_model=nli_model, llm_tokenizer=tokenizer)

args_generate = {"generation_config" : generation_config,
                 "max_new_tokens": 30}

estimator = ClaimConditionedProbability()

data_loader = DataLoader(chat_messages, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)

for batch in data_loader:
    encoded = tokenizer(batch, padding=True, return_tensors="pt")
    out = model_wrapper(encoded, model, args_generate)
    out.update(calc_nli(out))

    uncertianty_score = estimator(out)
    result = tokenizer.batch_decode(out['greedy_tokens'])

    for text, ue_score in zip(result, uncertianty_score):
        print("Output:", text)
        print("UE score:", ue_score)
        print()

/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. P

Output: A koala does not have a typical number of fingers like humans do. Instead, they have five non-opposable claws on each of
UE score: -0.05715579647124641

Output: The song "Yesterday" was written and first performed by the English singer-songwriter Paul McCartney. He wrote the song for the
UE score: -0.27115718924371196

Output: The song "Kukla Koldun" is a popular Russian children's song. The original version was recorded by the Soviet singer, Y
UE score: -0.023273841038365353

Output: In French, "I want a small cup of coffee" can be translated as "Je veux une tasse petite de café" or "
UE score: -0.2064187592371535

